In [185]:
pip install -r Project1/requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [186]:
import yfinance as yf
import plotly.graph_objs as go
import pandas as pd
import numpy as np

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Data downloading

In [187]:
def get_data(instrument: str,
             start_date: str,
             end_date: str,
             interval: str) -> pd.DataFrame:
    """
        Fetch historical market data from Yahoo Finance for a given instrument between the provided start and end dates at the given interval.
        The function returns a cleaned DataFrame with the data, excluding any missing values.

        Parameters:
        instrument (str): The ticker symbol of the instrument e.g. 'MSFT'.
        start_date (str): The start date for the data in 'YYYY-MM-DD' format.
        end_date (str): The end date for the data in 'YYYY-MM-DD' format.
        interval (str): The time interval between data points. Valid intervals are: ['1m', '2m', '5m', '15m', '30m', '60m', '90m', '1h', '1d', '5d', '1wk', '1mo'].

        Returns:
        pd.DataFrame: A Pandas DataFrame containing the historical market data for the given instrument, with any rows containing missing values (NaN) removed.
    """
    
    data = yf.download(tickers=instrument,
                       start=start_date,
                       end=end_date,
                       interval=interval)
    
    data = data.dropna(how='any')

    return data

# Indicators

## MACD

In [188]:
def macd(data: pd.DataFrame) -> pd.DataFrame:
    """
        Calculate the MACD (Moving Average Convergence Divergence) and generate buy/sell signals.

        Parameters:
        data (pd.DataFrame): DataFrame containing historical market data.

        Returns:
        pd.DataFrame: DataFrame with additional columns for 'MACD Buy Signal' and 'MACD Sell Signal', where a 1 indicates a signal and 0 means no signal.
    """

    data = data.copy()

    data['EMA12'] = data['Adj Close'].ewm(span=12, adjust=False).mean()
    data['EMA26'] = data['Adj Close'].ewm(span=26, adjust=False).mean()
    data['MACD'] = data['EMA12'] - data['EMA26'] # MACD Line
    data['Signal Line'] = data['MACD'].ewm(span=9, adjust=False).mean() # Signal Line

    data['MACD Buy Signal'] = 0
    data['MACD Sell Signal'] = 0

    for i in range(1, len(data)):
        # buy signal: MACD crosses above Signal Line
        if data['MACD'].iloc[i] > data['Signal Line'].iloc[i] and data['MACD'].iloc[i-1] <= data['Signal Line'].iloc[i-1]:
            data.loc[data.index[i], 'MACD Buy Signal'] = 1
        # sell signal: MACD crosses below Signal Line
        elif data['MACD'].iloc[i] < data['Signal Line'].iloc[i] and data['MACD'].iloc[i-1] >= data['Signal Line'].iloc[i-1]:
            data.loc[data.index[i], 'MACD Sell Signal'] = 1

    # drop intermediate columns
    data = data.drop(columns=['EMA12', 'EMA26'])

    return data

## RSI


In [189]:
def rsi(data: pd.DataFrame, period: int = 14) -> pd.DataFrame:
    """
    Calculate the Relative Strength Index (RSI)

    Parameters:
    data (pd.DataFrame): DataFrame containing historical market data.
    period (int): The period over which to calculate RSI, typically 14 days.

    Returns:
    pd.DataFrame: DataFrame with an additional 'RSI' column.
    """
    delta = data['Adj Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    
    rs = gain / loss
    data['RSI'] = 100 - (100 / (1 + rs))
    
    return data

## ATR

In [190]:
def atr(data: pd.DataFrame, period: int = 14) -> pd.DataFrame:
    """
    Calculate the Average True Range (ATR)

    Parameters:
    data (pd.DataFrame): DataFrame containing historical market data.
    period (int): The period over which to calculate ATR, typically 14 days.

    Returns:
    pd.DataFrame: DataFrame with an additional 'ATR' column.
    """
    high_low = data['High'] - data['Low']
    high_close = abs(data['High'] - data['Adj Close'].shift())
    low_close = abs(data['Low'] - data['Adj Close'].shift())
    
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    data['ATR'] = true_range.rolling(window=period).mean()
    
    return data

## PCA calculation

In [191]:
def apply_pca_trend_following(data: pd.DataFrame, indicators: list = ['MACD', 'RSI', 'ATR'],
                           n_components: int = 2, buy_threshold: float = 0.5, sell_threshold: float = -0.5) -> pd.DataFrame:
    """
    Apply PCA to reduce specified indicators to principal components and generate buy/sell signals.

    Parameters:
    data (pd.DataFrame): DataFrame containing calculated indicators.
    indicators (list): List of column names to include in PCA (default is ['MACD', 'RSI', 'ATR']).
    n_components (int): Number of principal components to keep (default is 2).
    buy_threshold (float): Threshold above which to generate a buy signal for PC1.
    sell_threshold (float): Threshold below which to generate a sell signal for PC1.

    Returns:
    pd.DataFrame: DataFrame with additional columns for 'PC1', 'PC2' (if n_components=2), 'PCA Buy Signal', and 'PCA Sell Signal'.
    """
    # Ensure that required indicators are in the DataFrame
    for indicator in indicators:
        if indicator not in data.columns:
            raise ValueError(f"Missing indicator {indicator} in data. Please calculate {indicator} first.")
    
    # Standardize the data for the selected indicators
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data[indicators])

    # Apply PCA
    pca = PCA(n_components=n_components)
    principal_components = pca.fit_transform(data_scaled)

    # Add the principal components to the DataFrame
    for i in range(n_components):
        data[f'PC{i+1}'] = principal_components[:, i]
    
    # Generate buy/sell signals based on the first principal component (PC1)
    data['PCA Buy Signal'] = 0
    data['PCA Sell Signal'] = 0

    for i in range(1, len(data)):
        # Buy Signal
        if data['PC1'].iloc[i] > buy_threshold:
            data.loc[data.index[i], 'PCA Buy Signal'] = 1
        # Sell Signal
        elif data['PC1'].iloc[i] < sell_threshold:
            data.loc[data.index[i], 'PCA Sell Signal'] = 1

    return data

In [192]:
def trend_following_strategy(data: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate buy/sell signals based on MACD, RSI, and ATR indicators for trend-following strategy.

    Parameters:
    data (pd.DataFrame): DataFrame containing historical market data with MACD, RSI, and ATR.

    Returns:
    pd.DataFrame: DataFrame with additional columns for 'Trend Buy Signal' and 'Trend Sell Signal'.
    """
    data = data.copy()
    
    data['Trend Buy Signal'] = 0
    data['Trend Sell Signal'] = 0

    for i in range(1, len(data)):
        # Buy signal: MACD bullish, RSI > 50, and ATR is high
        if (data['MACD'].iloc[i] > data['Signal Line'].iloc[i] and 
            data['RSI'].iloc[i] > 50 and 
            data['ATR'].iloc[i] > data['ATR'].mean()):
            data.loc[data.index[i], 'Trend Buy Signal'] = 1
        
        # Sell signal: MACD bearish, RSI < 50, and ATR is high
        elif (data['MACD'].iloc[i] < data['Signal Line'].iloc[i] and 
              data['RSI'].iloc[i] < 50 and 
              data['ATR'].iloc[i] > data['ATR'].mean()):
            data.loc[data.index[i], 'Trend Sell Signal'] = 1

    return data

In [193]:
strategies = {
    'MACD': macd,
    'PCA-trend-following' : apply_pca_trend_following,
    'Trend-Following': trend_following_strategy
}

In [194]:
def run_strategy(data: pd.DataFrame, strategy: str) -> pd.DataFrame:
    """
        Executes the specified trading strategy on the given data.

        Parameters:
        data (pd.DataFrame): DataFrame containing historical market data on which the strategy will be applied.
                             The DataFrame must have relevant columns for the selected strategy.
        
        strategy (str): The name of the strategy to be applied.

        Returns:
        pd.DataFrame: DataFrame with the strategy applied, including any newly added columns like buy/sell signals.
    """
    
    if strategy in strategies:
        data = strategies[strategy](data=data)
    else:
        print("Invalid strategy selected!")

    return data

# Plots

## Buy and sell signal plot

In [195]:
def plot_buy_sell_signal(data: pd.DataFrame,
                         instrument: str,
                         buy_signal_column: str,
                         sell_signal_column: str,
                         title: str):
    """
        Plots a price chart with buy and sell signals marked on it.

        Parameters:
        data (pd.DataFrame): A DataFrame containing historical market data. The DataFrame must have
                            at least the 'Adj Close' column and the specified buy/sell signal columns.
        
        instrument (str): The ticker symbol or name of the instrument being plotted.

        buy_signal_column (str): The column name containing the buy signals (e.g., 'MACD Buy Signal').
                                The function will mark buy signals where the value in this column is 1.

        sell_signal_column (str): The column name containing the sell signals (e.g., 'MACD Sell Signal').
                                The function will mark sell signals where the value in this column is 1.
        
        title (str): The title of the plot, which will be displayed on the chart after instrument name.
    """
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=data.index,
        y=data['Adj Close'],
        mode='lines',
        name='Price',
        line=dict(color='blue')
    ))

    # add buy signals to the price chart
    buy_signals = data[data[buy_signal_column] == 1].index
    fig.add_trace(go.Scatter(
        x=buy_signals,
        y=data.loc[buy_signals, 'Adj Close'],
        mode='markers',
        name=buy_signal_column,
        marker=dict(color='green', symbol='triangle-up', size=10)
    ))

    # add sell signals to the price chart
    sell_signals = data[data[sell_signal_column] == 1].index
    fig.add_trace(go.Scatter(
        x=sell_signals,
        y=data.loc[sell_signals, 'Adj Close'],
        mode='markers',
        name=sell_signal_column,
        marker=dict(color='red', symbol='triangle-down', size=10)
    ))

    fig.update_layout(
        title=f"{instrument} - {title}",
        xaxis_title="Date",
        yaxis_title="Price (USD)",
        xaxis_rangeslider_visible=False,
        hovermode="x unified",
    )

    fig.show()

# Performance Evaluation

## Win Rate

In [196]:
def calculate_win_rate(data: pd.DataFrame, 
                       buy_signal_column: str, 
                       sell_signal_column: str) -> float:
    """
        Calculates the win rate of a trading strategy based on buy and sell signals in the given data.

        This function simulates a trading strategy where a position is opened on a buy signal
        and closed on a sell signal assuming one signal at the time (if buy signal then wait for sell signal).

        Parameters:
        data (pd.DataFrame): A DataFrame containing market data, including the 'Adj Close' column
                            and the buy/sell signal columns specified by `buy_signal_column` and `sell_signal_column`.
        
        buy_signal_column (str): The name of the column containing buy signals (e.g., 'MACD Buy Signal').
                                A value of 1 in this column indicates a buy signal.
        
        sell_signal_column (str): The name of the column containing sell signals (e.g., 'MACD Sell Signal').
                                A value of 1 in this column indicates a sell signal.
    """

    open_position = False  # track if there is an open position
    total_trades = 0       # total number of trades (both wins and losses)
    wins = 0               # total number of winning trades

    for i in range(1, len(data)):
        if data[buy_signal_column].iloc[i] == 1 and not open_position: # buy
            open_position = True # open the position
            entry_price = data['Adj Close'].iloc[i]
            print(f"Buying  at {data.index[i]}: {entry_price:.2f}")
        elif data[sell_signal_column].iloc[i] == 1 and open_position: # sell
            exit_price = data['Adj Close'].iloc[i]
            profit_loss = exit_price - entry_price
            profit_loss_percent = (profit_loss / entry_price) * 100
            total_trades += 1  # increment total trades
            if profit_loss > 0: 
                wins += 1 # increment total wins
                print(f"Selling at {data.index[i]}: {exit_price:.2f} | Profit: {profit_loss_percent:.2f}%")
            else:
                print(f"Selling at {data.index[i]}: {exit_price:.2f} | Loss: {profit_loss_percent:.2f}%")
            open_position = False  # close the position
    
    if total_trades > 0:
        win_rate = wins / total_trades
    else:
        win_rate = 0

    win_rate = win_rate * 100

    print(f"Win Rate: {win_rate:.2f}%")


## Maximum DrawDown

In [197]:
#Load data
instrument='AAPL'
data = get_data(instrument=instrument, start_date='2000-01-01', end_date='2023-12-01', interval='1mo')
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-01-01,0.936384,1.084821,0.772321,0.926339,0.781411,1.255518e+10
2000-02-01,0.928571,1.070871,0.866071,1.023438,0.863318,7.319782e+09
2000-03-01,1.058594,1.342634,1.017857,1.212612,1.022896,8.698357e+09
2000-04-01,1.209821,1.245536,0.936384,1.107701,0.934398,8.662405e+09
2000-05-01,1.114955,1.127232,0.729911,0.750000,0.632660,9.807750e+09
...,...,...,...,...,...,...
2023-07-01,193.779999,198.229996,186.600006,196.449997,194.971786,9.960664e+08
2023-08-01,196.240005,196.729996,171.960007,187.869995,186.456345,1.322439e+09
2023-09-01,189.490005,189.979996,167.619995,171.210007,170.151169,1.337587e+09


## Sharpe Ratio

In [198]:
def calculate_sharpe_ratio(data, frequency):
    """
    Calculate Sharpe Ratio and Annualized Sharpe Ratio for given frequency.

    Parameters:
    - data (pd.DataFrame): Contains 'Adj Close' column.
    - frequency (str): 'monthly', 'daily', or 'hourly'.
    """
    rf_data = pd.read_csv('Project1/risk_free_rate.CSV')
    rf_data['Month'] = pd.to_datetime(rf_data['Month'], format='%Y%m')
    rf_data['RF'] = rf_data['RF'] / 100  # convert percentage to decimal
    rf_data.rename(columns={'RF': 'RF Monthly'}, inplace=True)
    rf_data = rf_data[['Month', 'RF Monthly']]

    data['Month'] = data.index.to_period('M').to_timestamp()
    data = pd.merge(data, rf_data, on='Month', how='left').set_index(data.index)
    data = data.drop(columns=['Month'])

    data['Periodic Return'] = data['Adj Close'].pct_change()

    # adjust risk-free rate based on frequency (no compounding)
    if frequency == 'daily':
        data['Risk-Free Rate'] = data['RF Monthly'] / 21 
        periods_per_year = 252
    elif frequency == 'hourly':
        data['Risk-Free Rate'] = data['RF Monthly'] / (21 * 6.5)
        periods_per_year = 252 * 6.5
    elif frequency == 'monthly':
        data['Risk-Free Rate'] = data['RF Monthly']
        periods_per_year = 12
    else:
        raise ValueError("Unsupported frequency. Use 'monthly', 'daily', or 'hourly'.")

    data['Excess Return'] = data['Periodic Return'] - data['Risk-Free Rate']
    mean_excess_return = data['Excess Return'].mean()
    std_return = data['Periodic Return'].std()

    sharpe_ratio = mean_excess_return / std_return

    annualized_sharpe_ratio = sharpe_ratio * np.sqrt(periods_per_year)

    # Print results
    print(f"Sharpe Ratio ({frequency}): {sharpe_ratio:.4f}")
    print(f"Annualized Sharpe Ratio: {annualized_sharpe_ratio:.4f}")


# Results

In [199]:
#Load data
instrument='AAPL'
data = get_data(instrument=instrument, start_date='2023-07-01', end_date='2023-12-01', interval='1d')

#Calculate strategy
data = macd(data)
data = rsi(data)
data = atr(data)
data = data.dropna()

data = run_strategy(data=data, strategy='Trend-Following')

#plot
plot_buy_sell_signal(data=data, instrument=instrument, buy_signal_column='Trend Buy Signal', sell_signal_column='Trend Sell Signal', title = 'Price with MACD Buy/Sell Signals')
win_rate = calculate_win_rate(data=data, buy_signal_column='Trend Buy Signal', sell_signal_column='Trend Sell Signal')




clean_data = data[['Adj Close', 'Trend Buy Signal', 'Trend Sell Signal']].copy()

# Create the "Full Data" DataFrame with all calculated indicators and signals
full_data = data.copy()  # This contains all columns by default


[*********************100%***********************]  1 of 1 completed


Buying  at 2023-07-27 00:00:00: 191.77
Selling at 2023-08-04 00:00:00: 180.62 | Loss: -5.81%
Buying  at 2023-08-30 00:00:00: 186.49
Selling at 2023-09-12 00:00:00: 175.21 | Loss: -6.05%
Buying  at 2023-11-07 00:00:00: 180.70
Win Rate: 0.00%


In [200]:
print(clean_data)
print(full_data)

             Adj Close  Trend Buy Signal  Trend Sell Signal
Date                                                       
2023-07-20  191.676773                 0                  0
2023-07-21  190.495728                 0                  0
2023-07-24  191.299622                 0                  0
2023-07-25  192.163055                 0                  0
2023-07-26  193.036438                 0                  0
...                ...               ...                ...
2023-11-24  189.043854                 0                  0
2023-11-27  188.864731                 0                  0
2023-11-28  189.471756                 0                  0
2023-11-29  188.446793                 0                  0
2023-11-30  189.023972                 0                  0

[94 rows x 3 columns]
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-07-20  195.089996  196.470001  192.500000  1

In [201]:
calculate_sharpe_ratio(data=data, frequency='daily')

Sharpe Ratio (daily): -0.0219
Annualized Sharpe Ratio: -0.3479
